In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
[sv, frqNum, dne,
tk, tb, health, age, flags,
rx, ry, rz, vx, vy, vz, wx, wy, wz,
tauSys, tau, gamma] = [22, -3, 747, 45000, 45900, 0, 0, 141, 
                      23553.6318359375, 9637.99951171875, 2603.29443359375,
                      0.404089927673340,  -0.027100563049316,  -3.548269271850586,
                      0.000000002793968, 0.000000002793968,  0.000000001862645,
                      -0.000000183470547,  -0.000110683962703, 0.000000000001819]
time = 45468
PDj = 0.072475022502641

Tj = (time - PDj) % 604800

T_mdv = Tj + tau - gamma * (Tj - tb) + tauSys

mu = 398600.44
Re = 6378.136
C20 = -1082.63e-6
Wz = 0.7292115e-4#*180/(2*np.pi)

In [3]:
def rungekut(s,h):
    arg = s
    k1 = h * f(arg)
    print('k1 ', k1)
    arg = s + 0.5 * k1
    k2 = h * f(arg)
    print('k2 ', k2)
    arg = s + 0.5 * k2
    k3 = h * f(arg)
    print('k3 ', k3)
    arg = s + k3
    k4 = h * f(arg)
    print('k4 ', k4)
    ds = (1/6)*(k1 + 2*k2 + 2*k3 + k4)
    print('ds ', ds, '\n')
    return s+ds  

def f(arg):
    r = np.sqrt(arg[0]*arg[0] + arg[1]*arg[1] + arg[2]*arg[2])
    A = mu/(r**3)
    return np.array([
    arg[3],
    arg[4],
    arg[5],
    (Wz**2 - A)*arg[0] + 2*Wz*arg[4] + (3/2)*C20*((mu*Re**2)/r**5)*arg[0]*(1 - (5*arg[2]**2)/r**2) + wx,
    (wz**2 - A)*arg[1] - 2*Wz*arg[3] + (3/2)*C20*((mu*Re**2)/r**5)*arg[1]*(1 - (5*arg[2]**2)/r**2) + wy,
    (-A)*arg[2] +                      (3/2)*C20*((mu*Re**2)/r**5)*arg[2]*(3 - (5*arg[2]**2)/r**2) + wz  
]) 

In [7]:
h = 60

In [9]:
f()

array([ 4.04089928e-01, -2.71005630e-02, -3.54826927e+00, -5.64778959e-04,
       -2.88418487e-04, -6.19970110e-05])

In [11]:
arg = np.array([rx, ry, rz, vx, vy, vz])

In [12]:
k1 = h * f(arg)

In [13]:
k1

array([ 2.42453957e+01, -1.62603378e+00, -2.12896156e+02, -3.38867375e-02,
       -1.73051092e-02, -3.71982066e-03])

In [ ]:
if T_mdv < 0:
    T_mdv += 86400

S,tt = [[rx, ry, rz, vx, vy, vz]],[]
#print(S)

t_s,t_e = tb, T_mdv
if tb < T_mdv:
    h = 50
else:
    h = -50
    
while t_s != t_e:
    if abs(t_e - t_s) < abs(h):
        h = t_e - t_s
    t_s += h
    tt.append(h)
    S.append([*rungekut(np.array(S[-1]),h)])
    #print(S[-1])

X_,Y_,Z_,vX,vY,vZ = S[-1]

In [ ]:
# def StatusGPS(sys, sys0,  x0 = 0, y0 = 0, z0 = 0):
#     Status = []
#     for epoch in tnrange(len(sys)):
#         stat = []
#         #rt = int((sys[epoch]['RT'].tod[0]*1e-3)%86400)
#         rt = sys[epoch]['RT']
#         pv = B1[epoch]['receiver']
#         sats = sys0[epoch]#['pseudo_params']
#         for sat in sats:
#             if sat.sv in EphemerisGE.keys():
#                 cur_eph = EphemerisGE[sat.sv]
#                 X,Y,Z = Uchet(*gps_coord(rt, cur_eph, sat.pr), pv.x, pv.y, pv.z)
#                 az, el = np.arctan2(X,Y)*180/np.pi, np.arctan2(Z,Y)*180/np.pi
#                 R = fR( X-(pv.x+x0), Y-(pv.y+y0), Z-(pv.z+z0) )
#                 stat.append(status(rt,sat.sv,X,Y,Z,pv.x+x0,pv.y+y0,pv.z+z0, R,(sat.cp),sat.pr, az, el))
#             else:
#                 stat.append(status(rt,sat.sv,0,0,0,pv.x+x0,pv.y+y0,pv.z+z0, 0,(sat.cp),sat.pr, 0, 0))
#         Status.append({rec.sv: rec for rec in stat})
#     return Status

# def StatusGLN(sys, sys0,  x0 = 0, y0 = 0, z0 = 0):
#     Status = []
#     for epoch in tnrange(len(sys)):
#         stat = []
#         #rt = int((sys[epoch]['RT'].tod[0]*1e-3)%86400)
#         rt = sys[epoch]['RT']
#         pv = B1[epoch]['receiver']
#         sats = sys0[epoch]#['pseudo_params']
#         for sat in sats:
#             if sat.sv in EphemerisNE.keys():
#                 cur_eph = EphemerisNE[sat.sv]
#                 X,Y,Z = Uchet(*gln_coord(rt, cur_eph, sat.pr), pv.x, pv.y, pv.z)
#                 az, el = np.arctan2(X,Y)*180/np.pi, np.arctan2(Z,Y)*180/np.pi
#                 R = fR( X-(pv.x+x0), Y-(pv.y+y0), Z-(pv.z+z0) )
#                 stat.append(status(rt,sat.sv,X,Y,Z,pv.x+x0,pv.y+y0,pv.z+z0, R,(sat.cp),sat.pr, az, el))
#             else:
#                 stat.append(status(rt,sat.sv,0,0,0,pv.x+x0,pv.y+y0,pv.z+z0, 0,(sat.cp),sat.pr, 0, 0))
#         Status.append({rec.sv: rec for rec in stat})
#     return Status

In [2]:
import win32com.client

h = win32com.client.Dispatch('matlab.application')

In [3]:
h.Execute ("plot([0 18], [7 23])")

''

In [5]:
h.Execute("Exam_Cholessky")

"??? Undefined function or variable 'Exam_Cholessky'.\n\n"

In [7]:
import matlab.engine

ModuleNotFoundError: No module named 'matlab'

In [ ]:
# Rc = lambda epoch, sv: Rgps[epoch][sv].R
# Rb = lambda epoch, sv: Bgps[epoch][sv].R
# pa = lambda epoch, sv: Rgps[epoch][sv].pr*C
# pb = lambda epoch, sv: Bgps[epoch][sv].pr*C
# fia = lambda epoch, sv: Rgps[epoch][sv].cp
# fib = lambda epoch, sv: Bgps[epoch][sv].cp
# qfia = lambda ts, te, sv : fia(te, sv) - fia(ts, sv)
# qfib = lambda ts, te, sv : fib(te, sv) - fib(ts, sv)

# dp = lambda sv, epoch: pa(epoch, sv) - pb(epoch, sv)
# dfi = lambda sv, epoch: fia(epoch, sv) - fib(epoch, sv)
# dqfi = lambda sv, epoch: qfia(ts, te, sv) - qfib(ts, te, sv)
# dR = lambda sv, epoch: Rc(epoch, sv) - Rb(epoch, sv) - Rc(epoch, el) + Rb(epoch, el)
# dqR = lambda sv, te, ts: Rc(te, sv) - Rb(te, sv) - Rc(ts, sv) + Rb(ts, sv)


# hx =  lambda epoch, sv: (Rgps[epoch][sv].X -  Rgps[epoch][sv].x) / Rc(epoch, sv)
# hy =  lambda epoch, sv: (Rgps[epoch][sv].Y -  Rgps[epoch][sv].y) / Rc(epoch, sv)
# hz =  lambda epoch, sv: (Rgps[epoch][sv].Z -  Rgps[epoch][sv].z) / Rc(epoch, sv)

# # d = figure(width = 800, plot_height = 200,)
# # d.line(Range, [dR(sv, ep)  - ddp[sv] for ep in Range])
# # d.line(Range, [dqR(sv, ep) - dqR(el, ep) - ddqfi[sv] for ep in Range])
# # d.line(Range, [dR(sv, ep) / lam - ddfi[sv] for ep in Range])
# # show(d)

# ddqfi = {sv: lam*(dqfi(sv, te) - dqfi(el, te)) for sv in sv_gps0}

# ddfi = {sv: dfi(sv, te) - dfi(el, te) for sv in sv_gps0}

# ddp = {sv: dp(sv, te) - dp(el, te) for sv in sv_gps0}

# Yp_GP = np.matrix([dR(sv, te)  - ddp[sv] for sv in sv_gps0 if sv!= el]).T

# HYp = np.matrix([(hx(te, sv) - hx(te, el),
#                    hy(te, sv) - hy(te, el),
#                    hz(te, sv) - hz(te, el)) for sv in sv_gps0 if sv != el])

# Yqfi_GP = np.matrix([dqR(sv, te, ts) - dqR(el, te, ts) - ddqfi[sv] for sv in sv_gps0 if sv!= el]).T

# HYqfix = [ hx(te, sv) - hx(ts, sv) - hx(te, el) + hx(ts, el)  for sv in sv_gps0 if sv != el]
# HYqfiy = [ hy(te, sv) - hy(ts, sv) - hy(te, el) + hy(ts, el)  for sv in sv_gps0 if sv != el]
# HYqfiz = [ hz(te, sv) - hz(ts, sv) - hz(te, el) + hz(ts, el)  for sv in sv_gps0 if sv != el]

# HYqfi = np.matrix([*zip(HYqfix, HYqfiy, HYqfiz)])

# fi_GP =  np.matrix([dR(sv, te) / lam - ddfi[sv] for sv in sv_gps0 if sv!= el]).T

# Hfi_GP = HYp / lam

# Y_GP = np.concatenate((Yp_GP, Yqfi_GP), axis = 0)
# H_GP = np.concatenate((HYp, HYqfi, Hfi_GP), axis = 0)

# ddx, ddy, ddz = Rgps[te][sv].x - Bgps[te][sv].x, Rgps[te][sv].y - Bgps[te][sv].y, Rgps[te][sv].z - Bgps[te][sv].z

# dTetta = np.matrix([ddx, ddy, ddz]).T

# sv_gps0

# print(np.abs(Y_GP - (H_GP * dTetta)[:2*(JGP-1)])[-(JGP-1):])

# print(np.abs( fi_GP - Hfi_GP * (dTetta) ))